# Vibe Coding with Databricks Assistant Lab 
## Notebook 1 - Chat Assistant for Grocery Sales ETL

Welcome to Notebook 1 of the Vibe Coding with Databricks Assistant Lab. Throughout this lab tutorial our goal will be to build a Demand Sensing engine using the power of Databricks! In this session, you will explore the chat functionality of the Databricks Assistant. You will use the Assistant to ask questions, write code, and debug individual cells as you build an ETL pipeline for a demand sensing demo. This hands-on lab will help you become familiar with interactive coding and troubleshooting in Databricks notebooks, leveraging the power of AI to accelerate your workflow.

This notebook leverages a Serverless DBSQL Warehouse compute. Please ensure you are connected to a DBSQL Warehouse by selecting connect in the top right, then selecting "Starter Warehouse" or similar warehouse designated for this lab

![sql warehouse](./includes/1.0_dbsql_warehouse.png)

In [0]:
%sql
-- Declare session variables and set defaults
DECLARE VARIABLE catalog_name STRING DEFAULT '<devsecops_labs>';
DECLARE VARIABLE schema_name  STRING DEFAULT '<demand_sensing>';
DECLARE VARIABLE volume_name  STRING DEFAULT 'data';
DECLARE VARIABLE file_name    STRING DEFAULT 'your_file.csv';

-- Switch catalog and schema using variables
USE CATALOG IDENTIFIER(catalog_name);
USE SCHEMA  IDENTIFIER(schema_name);

## Part 1. The Data

Our datasets include sales, product, store, and competitive pricing for a ficticious retail grocery company. We'll assume a process has been setup which stages files to our data lake and the data exists in its raw form as CSV files in a Unity Catalog Volume. Our four datasets are as follows:

* **Sales** - daily line item level sales transactions from the stores. Path: /Volumes/lp_dev/vibe_code_assistant_lab/data/raw/sales/
* **Products** - basic product dimensional data containing name, category, including brand, and unit price. Path: /Volumes/lp_dev/vibe_code_assistant_lab/data/raw/products/
* **Stores** - basic store dimensional data containing name, region, location. Path: /Volumes/lp_dev/vibe_code_assistant_lab/data/raw/stores/
* **Competitor Pricing** - daily product pricing data scraped from various competitor grocers. Path: /Volumes/lp_dev/vibe_code_assistant_lab/data/raw/competitor_pricing/

Before building an ETL pipeline, lets read some of the data and introduce you to the Databricks Assistant.

In [0]:
%sql 
SELECT * FROM read_files(
concat('/Volumes/', catalog_name, '/', schema_name, '/data/raw/sales'),
format => 'csv',
header => true
) 
LIMIT 10

In [0]:
%sql 
SELECT * FROM read_files(
concat('/Volumes/', catalog_name, '/', schema_name, '/data/raw/competitor_pricing'),
format => 'csv',
header => true
)
LIMIT 10

## Part 2. Databricks Assistant
<p align="center">
  <img src="https://www.databricks.com/sites/default/files/2024-06/db-assistant-header-graphic.png?v=1717414173" alt="Databricks Assistant" width="200"/>
</p>

[Databricks Assistant](https://docs.databricks.com/aws/en/notebooks/databricks-assistant-faq) is an AI-based pair-programmer and support agent that makes you more efficient as you create notebooks, queries, dashboards, and files. It can help you rapidly answer questions by generating, optimizing, completing, explaining, and fixing code and queries. We will focus on the Assistant abilities through the notebook interface but do know that the Assistant is available in a variety of Databricks interfaces including Dashboards, SQL Editor, and others. 

In its own words..

_The main features and modes of Databricks Assistant are:_

_Features:_

* _Generates, optimizes, completes, explains, and fixes code and queries._
* _Provides coding and debugging help, including quick fixes and error diagnosis._
* _Offers inline suggestions and autocomplete in notebooks._
* _Assists with data exploration and visualization using natural language._
* _Leverages Unity Catalog metadata for personalized, context-aware responses._
* _Supports filtering and exploring data with natural language prompts._
* _Can help diagnose errors in Databricks Jobs_

_Modes:_

* _Chat mode: Interact conversationally to get answers, code, and explanations._
* _Edit mode: Suggests changes across multiple notebook cells from a single prompt._
* _Agent mode: Automates multi-step data science workflows in notebooks and the SQL editor_

_You can also use /slash commands for common prompts and work with the Assistant directly in notebook cells_

## Part 3. Building an ETL Pipeline

### 3A. Using the Assistant for Bronze Layer Ingestion
We will start with using the Assistant in-line within a Notebook code cell. In this scenario, the Assistant's context is scoped to the cell it is working on. Other variations of the Assistant have a broader scope across an entire notebook or globally across your entire workspace, Unity Catalog, and Databricks documentation. 

1. Lets get started in the cell below by invoking the assistant either by clicking the **generate** text, or selecting the Assistant icon in the top right corner of the cell
![Assistant in Cell](./includes/1.1_assistant_in_cell.png)

2. Enter "Create a sales_bronze streaming table from the csvs in /Volumes/lp_dev/vibe_code_assistant_lab/data/raw/sales/ using SQL" and select the blue **Generate** button. 
![Assistant Generate](./includes/1.2_assistant_in_cell_generate.png)


3. Select **Accept** to accept the generated code and run the code cell. 
![Assistant Generate](./includes/1.3_assistant_in_cell_accept.png)

The correct SQL is pasted below, should the Assistant provide a different result:

```spark.sql("""
CREATE OR REFRESH STREAMING TABLE sales_bronze
AS SELECT *
FROM STREAM read_files(
  '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/sales/',
  format => 'csv',
  header => 'true'
)
""")



^^^ Use the cell above for generating code with the Assistant

In [0]:
-- CREATE OR REFRESH STREAMING TABLE sales_bronze
-- AS SELECT * FROM STREAM read_files(
--   '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/sales/',
--   format => 'csv',
--   header => true
-- );

In [0]:
CREATE OR REFRESH STREAMING TABLE competitor_pricing_bronze
AS SELECT * FROM STREAM read_files(
  '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/competitor_pricing/',
  format => 'csv',
  header => true
);

### 3B. The Global Assistant
Great we've ingested two tables and seen the power of code generation in a cell. We will now introduce the global assistant functionality. From the top right corner of the Databricks UI, select the symbol for the Databricks Assistant. 

This interface of the Assistant is scoped globablly, meaning it has context across your entire Databricks experience including notebooks, Unity Catalog objects, general Databricks documentation, best practices, and more, all while respecting permissioning set through Unity Catalog and workspace ACLs. 

With the Assistant open, notice we are using the "Chat" mode which provides a single response per prompt. Use this mode for basic Q&A, cell generation, and debugging a single cell statement. We'll cover the other modes in additional tutorial notebooks but for now try experimenting by asking some general Databricks Q&A such as:

* tell me about the medallion ETL model
* What are the CSV parameters for the read_files function?
* Why are streaming tables a strong solution for bronze level ingestion?

<p align="center">
  <img src="./includes/1.4_global_assistant.png" alt="Databricks Assistant" width="400"/>
</p>

So general questions and basic research are easily answered via the Assistant but LLMs only become more useful when grounded in your company's context and data. We can use the **@** symbol to add contextual objects from Unity Catalog. 

Try the asking the following statement of the assistant (Note: copy and paste is a finicky with [@] context clues and [/] slash commands so do try to write out the statement in the box):

* Tell me any quality issues you foresee with @sales_bronze and use cases or insights it may be helpful with


<p align="center">
  <img src="./includes/1.5_context_clues.png" alt="Databricks Assistant" width="600"/>
</p>

### 3C. Building a Transformed Silver Layer

Now that we have our two daily datasets ingested as bronze tables and been introduced to our handy Assistant, we will continue with our ETL pipeline by transforming our tables in the Silver layer. Inspect the table below and see if you can identify some table quality issues like the Assistant did in the previous step.

In [0]:
SELECT * FROM sales_bronze

Many columns have records with null values and data type mismatches. Run the CTAS statement below, then use the Assistant for inserting records into the silver table.


In [0]:
CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.sales_silver (
  transaction_id STRING,
  line_number INT,
  store_id STRING,
  product_id STRING,
  quantity INT,
  unit_price DOUBLE,
  discount DOUBLE,
  total_amount DOUBLE,
  payment_type STRING,
  channel STRING,
  loyalty_id STRING,
  transaction_ts TIMESTAMP,
  event_date DATE
)

With our silver table created, again open the assistant and copy in the following statement to generate the transformation code. You may need to re-write the two @table context clues as they do not usually copy in well. Once the Assistant generates the code, copy and paste the code into the following cell. Correct code is added below in case the Assistant does not provide a reasonable result.

```
Insert records from @sales_bronze into @sales_silver with the following rules:
Drop rows which do not match the schema. 
Drop rows with null values in transaction_ts, event_date, or quantity. 
Calculate null values in unit_price, discount, and total_amount given the formula: 
quantity * (unit_price - discount) = total_amount
```


<p align="center">
  <img src="./includes/1.6_silver_code_generation.png" alt="Databricks Assistant" width="600"/>
</p>

In [0]:
-- INSERT INTO lp_dev.vibe_code_assistant_lab.sales_silver
-- SELECT
--   CAST(transaction_id AS STRING) AS transaction_id,
--   CAST(line_number AS INT) AS line_number,
--   CAST(store_id AS STRING) AS store_id,
--   CAST(product_id AS STRING) AS product_id,
--   CAST(quantity AS INT) AS quantity,
--   -- Impute unit_price if null
--   COALESCE(
--     CAST(unit_price AS DOUBLE),
--     CASE
--       WHEN CAST(total_amount AS DOUBLE) IS NOT NULL
--         AND CAST(discount AS DOUBLE) IS NOT NULL
--         AND CAST(quantity AS DOUBLE) IS NOT NULL
--         THEN CAST(total_amount AS DOUBLE) / CAST(quantity AS DOUBLE) + CAST(discount AS DOUBLE)
--       ELSE NULL
--     END
--   ) AS unit_price,
--   -- Impute discount if null
--   COALESCE(
--     CAST(discount AS DOUBLE),
--     CASE
--       WHEN CAST(total_amount AS DOUBLE) IS NOT NULL
--         AND CAST(unit_price AS DOUBLE) IS NOT NULL
--         AND CAST(quantity AS DOUBLE) IS NOT NULL
--         THEN CAST(unit_price AS DOUBLE) - (CAST(total_amount AS DOUBLE) / CAST(quantity AS DOUBLE))
--       ELSE NULL
--     END
--   ) AS discount,
--   -- Impute total_amount if null
--   COALESCE(
--     CAST(total_amount AS DOUBLE),
--     CASE
--       WHEN CAST(unit_price AS DOUBLE) IS NOT NULL
--         AND CAST(discount AS DOUBLE) IS NOT NULL
--         AND CAST(quantity AS DOUBLE) IS NOT NULL
--         THEN CAST(quantity AS DOUBLE) * (CAST(unit_price AS DOUBLE) - CAST(discount AS DOUBLE))
--       ELSE NULL
--     END
--   ) AS total_amount,
--   CAST(payment_type AS STRING) AS payment_type,
--   CAST(channel AS STRING) AS channel,
--   CAST(loyalty_id AS STRING) AS loyalty_id,
--   CAST(transaction_ts AS TIMESTAMP) AS transaction_ts,
--   CAST(event_date AS DATE) AS event_date
-- FROM lp_dev.vibe_code_assistant_lab.sales_bronze
-- WHERE
--   transaction_ts IS NOT NULL
--   AND event_date IS NOT NULL
--   AND quantity IS NOT NULL
--   AND CAST(transaction_id AS STRING) IS NOT NULL
--   AND CAST(line_number AS INT) IS NOT NULL
--   AND CAST(store_id AS STRING) IS NOT NULL
--   AND CAST(product_id AS STRING) IS NOT NULL

We will follow the same pattern with competitor pricing data

In [0]:
CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.competitor_pricing_silver (
  competitor STRING,
  product_id STRING,
  price DOUBLE,
  scrape_ts TIMESTAMP,
  event_date DATE
)

In [0]:
INSERT INTO lp_dev.vibe_code_assistant_lab.competitor_pricing_silver
SELECT
  CAST(competitor AS STRING) AS competitor,
  CAST(product_id AS STRING) AS product_id,
  CAST(price AS DOUBLE) AS price,
  CAST(scrape_ts AS TIMESTAMP) AS scrape_ts,
  CAST(event_date AS DATE) AS event_date
FROM lp_dev.vibe_code_assistant_lab.competitor_pricing_bronze
WHERE
  competitor IS NOT NULL
  AND product_id IS NOT NULL
  AND price IS NOT NULL
  AND scrape_ts IS NOT NULL
  AND event_date IS NOT NULL
  AND _rescued_data IS NULL
;

And now create the Products dimension table, although there might be a snag...

In [0]:
CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.products
AS
SELECT *
FROM read_files(
  '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/products/',
  format => 'csv',
  headers => true,
  schema => 'product_id STRING, sku STRING, name STRING, category STRING, brand STRING, base_price DOUBLE, unit STRING'
);


When running the above cell a hidden syntax error will cause the following error within the cell.

![Assistant Generate](./includes/1.7_cell_error.png)

Notice the Assistant will try an inline correction directly to our cell as well as provide an option to "Diagnose Error". Select Diagnose Error to get a deeper understanding of our error - this one is a simple syntax error but the Assistant is trained to understand stack traces and common remediations for more complex errors. Select "Accept & Run" to correct the error.


In [0]:
-- CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.products AS
-- SELECT *
-- FROM read_files(
--   '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/products/',
--   format => 'csv',
--   header => true,
--   schema => 'product_id STRING, sku STRING, name STRING, category STRING, brand STRING, base_price DOUBLE, unit STRING'
-- );

Create the Stores dimension table

In [0]:
CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.stores AS
SELECT *
FROM read_files(
  '/Volumes/lp_dev/vibe_code_assistant_lab/data/raw/stores/',
  format => 'csv',
  header => true,
  schema => 'store_id STRING, name STRING, region STRING, city STRING, state STRING, zipcode STRING, opened_date DATE'
);

### 3D. Gold Feature Set Table

With our silver transformation layer created, lets now combine all of these tables into a feature set data product for our Data Scientist to leverage for the demand sensing engine.


`Created a feature set for sales demand sensing by joining  @sales_silver , @competitor_pricing_silver , @products , and @stores.`

![Gold Table Prompt](./includes/1.8_gold_table.png)

Then accept the code and run the cell.

![Gold table CTAS statement](./includes/1.9_gold_accept.png)

In [0]:
-- CREATE OR REPLACE TABLE lp_dev.vibe_code_assistant_lab.sales_demand_sensing_gold AS
-- SELECT
--   s.transaction_id,
--   s.line_number,
--   s.store_id,
--   s.product_id,
--   s.quantity,
--   s.unit_price,
--   s.discount,
--   s.total_amount,
--   s.payment_type,
--   s.channel,
--   s.loyalty_id,
--   s.transaction_ts,
--   s.event_date,
--   p.sku,
--   p.name AS product_name,
--   p.category,
--   p.brand,
--   p.base_price,
--   p.unit AS product_unit,
--   st.store_name,
--   st.store_manager_id,
--   st.address,
--   st.city AS store_city,
--   st.state AS store_state,
--   cp.competitor,
--   cp.price AS competitor_price,
--   cp.scrape_ts AS competitor_scrape_ts,
--   cp.event_date AS competitor_event_date
-- FROM lp_dev.vibe_code_assistant_lab.sales_silver s
-- LEFT JOIN lp_dev.vibe_code_assistant_lab.products p
--   ON s.product_id = p.product_id
-- LEFT JOIN main.tls_telco_stores.stores st
--   ON s.store_id = st.store_id
-- LEFT JOIN lp_dev.vibe_code_assistant_lab.competitor_pricing_silver cp
--   ON s.product_id = cp.product_id
--   AND s.event_date = cp.event_date
;

Congratulations, you have reached the end of notebook 1. Lets continue on to Notebook 2 to and continue to understand our Assistant helper's capabilities.
